<a href="https://colab.research.google.com/github/CatS1mp/NLP---Huggin-Face/blob/main/NLP_Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CHƯƠNG 6: THE TOKENIZERS LIBRARY**#

##**6.1. Chuẩn bị dữ liệu và tạo tokenizer mới**

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("code_search_net", "python", split="train")

def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["whole_func_string"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Khởi tạo tokenizer với BPE
tokenizer = Tokenizer(models.BPE())

# Thiết lập pre-tokenizer (ByteLevel cho GPT-2)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# Huấn luyện tokenizer
special_tokens = ["<|endoftext|>"]
trainer = trainers.BpeTrainer(vocab_size=52000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

##**6.2. Thêm hậu xử lý và giải mã**

In [ ]:
from tokenizers import processors

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

In [ ]:
from tokenizers import decoders

tokenizer.decoder = decoders.ByteLevel()

In [ ]:
encoding = tokenizer.encode("def add_numbers(a, b):")
print(encoding.tokens)  # Ví dụ: ['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):']
print(tokenizer.decode(encoding.ids))  # "def add_numbers(a, b):"

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):']
def add_numbers(a, b):


##**6.3. Lưu và tải tokenizer**

In [ ]:
tokenizer.save("my_tokenizer.json")

In [ ]:
new_tokenizer = Tokenizer.from_file("my_tokenizer.json")

##**6.4. Sử dụng tokenizer với Transformers**

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

##**6.5. Xây dựng tokenizer cho BERT (WordPiece)**

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(), normalizers.Lowercase()])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
from tokenizers import processors

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

In [ ]:
from tokenizers import decoders

tokenizer.decoder = decoders.ByteLevel()

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers

# Tạo tokenizer với Unigram
tokenizer = Tokenizer(models.Unigram())
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

# Định nghĩa danh sách token đặc biệt
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]

# Thiết lập trainer
trainer = trainers.UnigramTrainer(vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>")

# Hàm giả lập dữ liệu đầu vào
def get_training_corpus():
    return [
        "<cls> Xin chào! Đây là một câu ví dụ. <sep>",
        "<s> Học sâu và NLP rất thú vị. </s>",
        "Tokenization là quan trọng trong NLP.",
        "Đây là một câu dài " + "a" * 600 + " để kiểm tra lỗi.",  # Quá dài
        "",  # Câu rỗng
        None  # Dữ liệu không hợp lệ
    ]

# Hàm giới hạn độ dài câu & kiểm tra dữ liệu hợp lệ
def limit_sequence_length(corpus_iterator, max_length=512):
    for sequence in corpus_iterator:
        if isinstance(sequence, str) and len(sequence) > 0:  # Chỉ chấp nhận chuỗi hợp lệ
            yield sequence[:max_length]  # Cắt nếu quá dài
        else:
            print(f"Skipping invalid sequence: {sequence}")  # Debug lỗi dữ liệu

# Lọc và giới hạn dữ liệu đầu vào
limited_corpus = limit_sequence_length(get_training_corpus())

# Huấn luyện tokenizer với dữ liệu đã lọc
tokenizer.train_from_iterator(limited_corpus, trainer=trainer)

# Kiểm tra tokenizer với một câu mẫu
print(tokenizer.encode("Học sâu và NLP rất thú vị.").tokens)


Skipping invalid sequence: 
Skipping invalid sequence: None
['▁', 'H', 'ọ', 'c', '▁', 's', 'â', 'u', '▁v', 'à', '▁NLP', '▁', 'r', 'ấ', 't', '▁', 't', 'h', 'ú', '▁v', 'ị', '.']
